## PyTorch层和参数基础

In [46]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.1332, -0.0452,  0.1903,  0.0644, -0.1475, -0.0119,  0.0314,  0.1750,
         -0.0359, -0.0328],
        [ 0.1034,  0.0574,  0.1422,  0.0808, -0.0259, -0.0198, -0.1130,  0.0627,
          0.0044,  0.2505]], grad_fn=<AddmmBackward0>)

### 自定义层

In [8]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        # nn.ReLU()是构造了一个ReLU对象，torch.nn.functional.ReLU()是函数调用
        return self.out(F.relu(self.hidden(X)))
net = MLP()
net(X)

tensor([[ 0.2339,  0.1686, -0.2848, -0.0901,  0.0667,  0.1549,  0.0866,  0.0973,
          0.0122, -0.0485],
        [ 0.2939, -0.0491, -0.2441, -0.0038,  0.0221,  0.0778,  0.1013,  0.1112,
          0.1418,  0.0740]], grad_fn=<AddmmBackward0>)

In [14]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
            
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1759, -0.2794, -0.0257,  0.1091, -0.1999, -0.4960, -0.1099,  0.1498,
         -0.1448, -0.0768],
        [ 0.0311, -0.2505,  0.0542,  0.0806, -0.1063, -0.2330,  0.0084,  0.0228,
         -0.0769,  0.0701]], grad_fn=<AddmmBackward0>)

#### 下面的例子是我们继承nn.Module可以在正向传播和初始化的时候做很多灵活的事情

In [16]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False, dtype=torch.float32)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

tensor(-0.2138, grad_fn=<SumBackward0>)

#### 各种模块混搭

In [20]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        X = self.linear(self.net(X))
        return X
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)


tensor(-0.0879, grad_fn=<SumBackward0>)

### 模型参数管理

In [39]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(1, 4)
net(X)

tensor([[-0.2302]], grad_fn=<AddmmBackward0>)

In [40]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.3534, -0.3325, -0.3122, -0.1845,  0.2951, -0.1779,  0.2312, -0.0008]])), ('bias', tensor([-0.0646]))])


In [41]:
print(net[2].bias)

Parameter containing:
tensor([-0.0646], requires_grad=True)


In [42]:
print(net[2].bias.data)

tensor([-0.0646])


In [43]:
print(net[2].weight.grad)

None


In [44]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [45]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [47]:
net.state_dict()['2.bias'].data

tensor([-0.0646])

### 从嵌套块收集参数

模块化的组装在打印和访问的时候都有优势

In [49]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
X = torch.rand(2, 4)
rgnet(X)

tensor([[0.1814],
        [0.1815]], grad_fn=<AddmmBackward0>)

In [50]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


### 内置初始化

In [53]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data, net[0].bias.data

(tensor([[ 0.0094,  0.0082,  0.0070, -0.0142],
         [-0.0109, -0.0180, -0.0057,  0.0047],
         [ 0.0069, -0.0088, -0.0087, -0.0046],
         [ 0.0022,  0.0113,  0.0058,  0.0081],
         [-0.0042,  0.0123, -0.0066, -0.0158],
         [-0.0151,  0.0255,  0.0098,  0.0166],
         [ 0.0054, -0.0173,  0.0142,  0.0061],
         [ 0.0059,  0.0064, -0.0031,  0.0327]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [54]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data, net[0].bias.data

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [55]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.1333,  0.5585,  0.5114,  0.4537],
        [-0.0538, -0.3174,  0.4085,  0.4626],
        [ 0.5604, -0.6939,  0.4160,  0.3096],
        [-0.6202, -0.4470, -0.0310,  0.4940],
        [-0.3542,  0.6822,  0.0160,  0.3630],
        [-0.0207,  0.6689,  0.6394,  0.3258],
        [-0.6445,  0.1882,  0.2007,  0.1279],
        [-0.3081, -0.6236,  0.2784,  0.0141]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [56]:
def my_init(m):
    if type(m) == nn.Linear:
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net[0].apply(my_init)
print(net[0].weight.data)

tensor([[-7.4807,  0.0000,  0.0000,  0.0000],
        [ 7.1709,  8.2088, -0.0000,  0.0000],
        [-8.2866, -7.0116,  5.9289, -0.0000],
        [ 0.0000,  8.9970,  0.0000, -7.6205],
        [ 6.8355, -7.8800,  9.3160,  9.0835],
        [-0.0000,  0.0000, -0.0000,  0.0000],
        [-0.0000, -8.8462, -0.0000, -7.2922],
        [ 5.9192,  0.0000,  6.3449,  7.3030]])


In [58]:
net[0].weight.data += 1
print(net[0].weight)

Parameter containing:
tensor([[-5.4807,  2.0000,  2.0000,  2.0000],
        [ 9.1709, 10.2088,  2.0000,  2.0000],
        [-6.2866, -5.0116,  7.9289,  2.0000],
        [ 2.0000, 10.9970,  2.0000, -5.6205],
        [ 8.8355, -5.8800, 11.3160, 11.0835],
        [ 2.0000,  2.0000,  2.0000,  2.0000],
        [ 2.0000, -6.8462,  2.0000, -5.2922],
        [ 7.9192,  2.0000,  8.3449,  9.3030]], requires_grad=True)


### 参数绑定

In [59]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 自定义层和参数

In [62]:
class MyLinear(nn.Module):
    def __init__(self, in_unit, unit):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_unit, unit))
        self.bias = nn.Parameter(torch.randn(unit,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[ 2.1682,  1.9794,  0.9286],
        [ 0.1717,  0.2689, -0.5663],
        [-0.0143,  0.9596, -0.6437],
        [-1.3514,  1.5717,  1.1344],
        [-1.0662,  0.6589,  0.2862]], requires_grad=True)

In [63]:
X = torch.rand(3, 5)
dense(X)

tensor([[0.7300, 3.6438, 0.2151],
        [1.3711, 2.8669, 0.0000],
        [1.7082, 2.7151, 0.0000]])

In [64]:
net = nn.Sequential(MyLinear(3, 5), MyLinear(5, 1))
net(torch.rand(2, 3))

tensor([[0.1737],
        [0.0000]])

### torch文件操作

In [68]:
x = torch.arange(4)
torch.save(x, './test-files/x-file')
y = torch.load('./test-files/x-file')
y

tensor([0, 1, 2, 3])

In [69]:
torch.save([x, y], './test-files/x-file')
a, b = torch.load('./test-files/x-file')
(a, b)

(tensor([0, 1, 2, 3]), tensor([0, 1, 2, 3]))

### 存一个模型

In [71]:
torch.save(net.state_dict(), './test-files/net-file')

In [2]:
clone = nn.Sequential(MyLinear(3, 5), MyLinear(5, 1))
clone.load_state_dict(torch.load('./test-files/net-file'))
clone.eval()
clone.weight.data == net.weight.data

NameError: name 'nn' is not defined